In [57]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

## Final Project Yuqi Liu

## Introdution

* **Data Source**: Text Data (unstructured data) scraped from Game Spot, a famous Game News Website
* **Purpose**: It is very costly to maintain exclusive game releases and intellectual property for a platform, so the games supported by each platform may vary from year to year. By analyzing the 2022 game release article, this project will update the game landscape so that gamers can select the most appropriate platform for the 2023 and discount seasons.
* **Question Answered by the project**: Which platforms Gamers should consider purchasing to play their desired games ?
* **Conclusion**: Despite dramatic industry mergers, there are still more games that can be played on both console and PC than on both consoles. 72% of games are supported by the PC, making it an excellent platform for the majority of gamers. Nonetheless, both PlayStation and Xbox have exclusive games. If you have a favorite IP or game developer, you may need to check the exclusive game list and purchase the required platform. The number of games between the older and newer console versions is relatively small. Players would be able to make a decision based on the trade-off between gaming experiences and price, with less need to consider game differences.

In [65]:
# Send a GET request to the website
url = "https://www.gamespot.com/articles/every-game-released-in-2022/1100-6499287/"
response = requests.get(url)

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.content, "html.parser")

# Find all the paragraphs in the HTML
paragraphs = soup.find_all("p")

# Create a list to store the scraped data
data = []

# Loop through each paragraph and find all the game titles
for p in paragraphs:
    for title in p.find_all("a"):
        for platform in p.find_all('em'):
            data.append({"title": title.get_text(),
                         "platform": platform.get_text().strip()})

# Create a data frame from the scraped data
df = pd.DataFrame(data)

# Split the platform column by comma and add a new row for each parted value
df_new = pd.DataFrame(df['platform'].str.split(', ').tolist(), index=df['title']).stack()
df_new = df_new.reset_index([0, 'title'])
df_new.columns = ['title', 'platform']

# Clean up the platform names
df_new['platform'] = df_new['platform'].str.strip('() ')

# Remove comma and parentheses from platform names
df_new['platform'] = df_new['platform'].str.replace(',', '').str.replace(r'\(|\)', '')

# Remove the '-' character from platform names
df_new['platform'] = df_new['platform'].str.replace('-', '')

# Adjust the platform name
df_new['platform'] = df_new['platform'].replace({
    'PC VR': 'PC',
    'Xbox One': 'Xbox One',
    'box One': 'Xbox One',
    'Xbox Series X/S|Windows': 'Xbox Series X/S',
    'Xbox Series X/S': 'Xbox Series X/S',
    'Xbox Series X': 'Xbox Series X/S',
    'Xbox Series X|S': 'Xbox Series X/S',
    'Nintendo Switch Nintendo Switch Switch': 'Nintendo Switch',
    'Nintendo': 'Nintendo Switch',
    'Nintendo Switch Switch': 'Nintendo Switch',
    'Nintendo Switch Switch  November 11': 'Nintendo Switch',
    'Switch': 'Nintendo Switch',
    'Switch  June 1 ': 'Nintendo Switch',
    'Switch': 'Nintendo Switch',
    'Switch  November 11': 'Nintendo Switch',
    'Switch PC': 'Nintendo Switch',
    'PlayStation 4': 'PS4',
    'PlayStation 5': 'PS5',
    'Apple Arcade': 'iOS',
    'Oculus Quest 2': 'VR',
    'Quest 2': 'VR',
    'PlayStation VR': 'VR'
    
})

# Count the number of unique titles for each platform
platform_counts = df_new.groupby('platform')['title'].nunique()

# Filter the platform counts where count >= 5. 
# There are a few mobile and VR games that I choose not to discuss in this project
platform_counts = platform_counts[platform_counts >= 5]

# Display the platform counts
print(platform_counts)

platform
Android             12
Mac                  5
Nintendo Switch    141
PC                 268
PS4                146
PS5                139
Stadia               8
VR                   5
Xbox One           142
Xbox Series X/S    147
iOS                 15
Name: title, dtype: int64


<ipython-input-65-7a425e34d2e9>:33: FutureWarning: The default value of regex will change from True to False in a future version.
  df_new['platform'] = df_new['platform'].str.replace(',', '').str.replace(r'\(|\)', '')


### The Cleaned data 

In [64]:
df_new

,title,platform
0,Monster Hunter Rise,PC
1,The Anacrusis,Xbox Series X/S
2,The Anacrusis,Xbox One
3,The Anacrusis,PC
4,God of War,PC
...,...,...
1063,Yu-Gi-Oh! Master Duel,Xbox One
1064,Yu-Gi-Oh! Master Duel,Nintendo Switch
1065,Yu-Gi-Oh! Master Duel,PC
1066,Yu-Gi-Oh! Master Duel,iOS


## Explortive Data Analysis

### Percentage of Games supported PC

In [46]:
268/374

0.7165775401069518

## Compare PlayStation and Xbox

### Number of games exclusive to Xbox One and Xbox Series X/S:

In [47]:
# Create a subset of the original DataFrame for Xbox One and Xbox Series X/S platforms
xbox_subset = df_new[df_new['platform'].isin(['Xbox One', 'Xbox Series X/S'])]

# Filter the subset to include only the games not available on PS5 or PS4
xbox_exclusive = xbox_subset[~xbox_subset['platform'].isin(['PS5', 'PS4'])]

# Count the number of unique titles in the filtered subset
xbox_exclusive_count = xbox_exclusive['title'].nunique()

# Print the result
print("Number of games exclusive to Xbox One and Xbox Series X/S:", xbox_exclusive_count)


Number of games exclusive to Xbox One and Xbox Series X/S: 169


### Names of the Exclusive Games on Xbox One and X/S

In [55]:
# Create a new dataframe with only Xbox One and Xbox Series X/S platforms
xbox_df = df_new[df_new['platform'].isin(['Xbox One', 'Xbox Series X/S'])]

# Filter out the games that are also available on PS4 or PS5 platforms
exclusive_games = xbox_df.loc[(~xbox_df['title'].isin(df_new[df_new['platform'].isin(['PS4', 'PS5'])]['title']))]['title']

exclusive_games = exclusive_games.drop_duplicates()
# Print out the names of the exclusive games
print(exclusive_games)

1                                          The Anacrusis
5                                 Nobody Saves the World
47                                            CrossfireX
132                                            Shredders
178                     Babylon's Fall: Ultimate Edition
232     Microsoft Flight Simulator: Top Gun Maverick DLC
262                         Fall Guys: Ultimate Knockout
265                                   Naraka: Bladepoint
314                                  PowerWash Simulator
324                                        As Dusk Falls
330                       Forza Horizon 5 Hot Wheels DLC
390                                    Tribes of Midgard
419                                               Islets
440                                          Immortality
479                                           Family Man
522                                      Slime Rancher 2
533                                             Grounded
569                            

### Number of Games Exclusive to PS5 and PS4:

In [49]:
# Create a subset of the original DataFrame for PS5 and PS4 platforms
xbox_subset = df_new[df_new['platform'].isin(['PS5', 'PS4'])]

# Filter the subset to include only the games not available on PS5 or PS4
xbox_exclusive = xbox_subset[~xbox_subset['platform'].isin(['Xbox One', 'Xbox Series X/S'])]

# Count the number of unique titles in the filtered subset
xbox_exclusive_count = xbox_exclusive['title'].nunique()

# Print the result
print("Number of games exclusive to PS 5 and PS 4:", xbox_exclusive_count)

Number of games exclusive to PS 5 and PS 4: 166


### Games Exclusive to PS5 and PS4:

In [56]:
# Create a new dataframe with only Xbox One and Xbox Series X/S platforms
PS_df = df_new[df_new['platform'].isin(['PS5', 'PS4'])]

# Filter out the games that are also available on Xbox One and Xbox Series X/S  platforms
exclusive_games = PS_df.loc[(~PS_df['title'].isin(df_new[df_new['platform'].isin(['Xbox One', 'Xbox Series X/S'])]['title']))]['title']

exclusive_games = exclusive_games.drop_duplicates()
# Print out the names of the exclusive games
print(exclusive_games)

24               Uncharted: Legacy of Thieves Collection
38                                                  Sifu
66                                Horizon Forbidden West
80                                                Monark
84                                            Sol Cresta
110                                       Babylon's Fall
113                                       Gran Turismo 7
127                                   Grand Theft Auto V
134                              Persona 4 Arena Ultimax
136                                     Anno: Mutationem
146                                     GhostWire: Tokyo
190                  Vampire: The Masquerade - Bloodhunt
205                                   Salt and Sacrifice
266                       Little Noah: Scion of Paradise
269                                             DNF Duel
327                                                Stray
361                                             GigaBash
377                            